In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import os
from os.path import exists
import time
from rtsvg import *
rt = RACETrack()
ofv = rt.ontologyFrameworkInstance()

In [ ]:
#
# Keep track of all emails seen
#
emails_seen = set()

#
# Email File First // file has utf8 issues :(
#
_bin_ = open('../../../data/2014_vast/MC1/email headers.csv', 'rb').read()
print(len(_bin_))
as_str = []
for i in range(len(_bin_)):
    c = chr(_bin_[i])
    if (c >= 'a' and c <= 'z') or (c >= 'A' and c <= 'Z') or (c >= '0' and c <= '9') or \
       c == ' ' or c == '.' or c == ',' or c == '_' or \
       c == '-' or c == '!' or c == '@' or c == '"' or \
       c == ':' or c == '\t' or c == '\n' or c == '\r' or \
       c == '/' or c == '?' or c == ')' or c == '(' or \
       c == "'":
        as_str.append(c)
    elif ord(c) == 146:
        as_str.append("'")
    else:
        print('"'+c+'"', ord(c))
_str_ = ''.join(as_str)
print(len(_str_))
open('../../../data/2014_vast/MC1/email_headers_fixed.csv', 'wt').write(_str_)
df_email = pl.read_csv('../../../data/2014_vast/MC1/email_headers_fixed.csv')
df_email.sample(3)

In [ ]:
#
# Ontology Based On:
# https://www.semanticdesktop.org/ontologies/2007/03/22/nmo/
#
_lu_ = {'__id_email__':     [],
        'timestamp':        [],
        'timestamp_str':    [],
        '__id_sender__':    [],
        'sender':           [],
        '__id_recipient__': [],
        'recipient':        [],
        'subject':          []}
for i in range(len(df_email)):
    _id_              = ofv.createId('nmo:Email')
    _fm_              = df_email['From'][i].strip().lower()
    emails_seen.add(_fm_)
    _fm_id_           = ofv.resolveUniqIdAndUpdateLookups(_fm_, 'nmo:EmailAddress', 'uniq', 'sbj,obj')
    _timestamp_str_   = df_email['Date'][i]
    _timestamp_       = str(pd.to_datetime(_timestamp_str_))
    #_ts_as_int_       = ofv.resolveUniqIdAndUpdateLookups(_timestamp_, 'xsd:dateTime', 'dttm', 'obj')
    _subject_         = df_email['Subject'][i].strip()
    #_subject_literal_ = ofv.resolveUniqIdAndUpdateLookups(_subject_, 'nmo:MessageHeader', 'cont', 'obj')
    #ofv.bufferTripleToAddLater(_id_, 'nmo:emailFrom',   _fm_id_)
    #ofv.bufferTripleToAddLater(_id_, 'nmo:sentDate',    _ts_as_int_)
    #ofv.bufferTripleToAddLater(_id_, 'nmo:headerValue', _subject_literal_)
    _to_              = df_email['To'][i]
    for _to_actual_ in _to_.split(','):
        _to_actual_ = _to_actual_.strip().lower()
        emails_seen.add(_to_actual_)
        _to_actual_id_ = ofv.resolveUniqIdAndUpdateLookups(_to_actual_, 'nco:EmailAddress', 'uniq', 'sbj,obj')
        #ofv.bufferTripleToAddLater(_id_, 'nmo:emailTo', _to_actual_id_)
        _lu_['__id_email__']    .append(_id_),           _lu_['timestamp'] .append(_timestamp_),    _lu_['timestamp_str'].append(_timestamp_str_)
        _lu_['__id_sender__']   .append(_fm_id_),        _lu_['sender']    .append(_fm_)
        _lu_['__id_recipient__'].append(_to_actual_id_), _lu_['recipient'] .append(_to_actual_)
        _lu_['subject']         .append(_subject_)
#ofv.appendBufferedTriplesAndClearBuffer()
#len(ofv.df_triples)
df_email_ont = pl.DataFrame(_lu_)
df_email_ont = rt.columnsAreTimestamps(df_email_ont, 'timestamp')
df_email_ont_mapping = [ ['@__id_email__', 'nmo:emailFrom',   '@__id_sender__'],
                         ['@__id_email__', 'nmo:sendDate',    '@timestamp'],
                         ['@__id_email__', 'nmo:headerValue', '@subject'],
                         ['@__id_email__', 'nmo:emailTo',     '@__id_recipient__'] ]
ofv.addTabularDataFrame(df_email_ont, df_email_ont_mapping)
df_email_ont.sample(3)

In [4]:
df_employee_recs = pl.read_excel('../../../data/2014_vast/MC1/EmployeeRecords.xlsx')

In [ ]:
def fixDate(_date_):
    if type(_date_) != str: return _date_
    _split_ = _date_.split('-')
    _year_, _month_, _day_ = int(_split_[2]), int(_split_[0]), int(_split_[1])
    if _year_ < 30: _year_ += 2000
    else:           _year_ += 1900
    _date_str_ = f'{_year_}-{_month_:02}-{_day_:02}'
    return _date_str_

#
# https://w3id.org/MON/person.owl
# https://www.w3.org/TR/vocab-org/
#
#
for i in range(len(df_employee_recs)):
    _id_ = ofv.createId('mon:Person')
    _last_, _first_, _dob_, _pob_, _gender_ = df_employee_recs['LastName'][i], df_employee_recs['FirstName'][i], df_employee_recs['BirthDate'][i], df_employee_recs['BirthCountry'][i], df_employee_recs['Gender'][i]
    _last_id_   = ofv.resolveUniqIdAndUpdateLookups(_last_,         'xsd:string', 'ambi', 'obj')
    _first_id_  = ofv.resolveUniqIdAndUpdateLookups(_first_,        'xsd:string', 'ambi', 'obj')
    _pob_id_    = ofv.resolveUniqIdAndUpdateLookups(_pob_,          'xsd:string', 'uniq', 'obj')
    _dob_id_    = ofv.resolveUniqIdAndUpdateLookups(fixDate(_dob_), 'xsd:date',   'date', 'obj')
    _gender_id_ = ofv.resolveUniqIdAndUpdateLookups(_gender_,       'mon:Sex',    'cata', 'obj')

    ofv.bufferTripleToAddLater(_id_, 'mon:hasBirthDate',  _dob_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:hasBirthPlace', _pob_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:hasGender',     _gender_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:lastName',      _last_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:firstName',     _first_id_)

    _employment_department_  = df_employee_recs['CurrentEmploymentType'][i]
    _employment_title_       = df_employee_recs['CurrentEmploymentTitle'][i]
    _employment_start_date_  = df_employee_recs['CurrentEmploymentStartDate'][i]

    _employment_id_          = ofv.createId('org:Employment')
    ofv.bufferTripleToAddLater(_id_, 'hasEmployment',  _employment_id_)

    _employment_department_id_    = ofv.resolveUniqIdAndUpdateLookups(_employment_department_,          'org:Organization',      'uniq',     'sbj,obj')
    _employment_title_id_         = ofv.resolveUniqIdAndUpdateLookups(_employment_title_,               'org:EmploymentTitle',   'ambi',     'sbj,obj')
    _employment_start_date_id_    = ofv.resolveUniqIdAndUpdateLookups(fixDate(_employment_start_date_), 'xsd:date',              'date',     'obj')

    ofv.bufferTripleToAddLater(_employment_id_, 'employedBy',          _employment_department_id_)
    ofv.bufferTripleToAddLater(_employment_id_, 'hasEmploymentTitle',  _employment_title_id_)
    ofv.bufferTripleToAddLater(_employment_id_, 'hasStartDate',        _employment_start_date_id_)

    _email_address_          = df_employee_recs['EmailAddress'][i].strip().lower()
    emails_seen.add(_email_address_)
    _email_address_id_       = ofv.resolveUniqIdAndUpdateLookups(_email_address_, 'nmo:EmailAddress', 'uniq', 'sbj,obj')

    ofv.bufferTripleToAddLater(_id_, 'nmo:hasEmailAddress', _email_address_id_)

ofv.appendBufferedTriplesAndClearBuffer()
len(ofv.df_triples)

In [6]:
ln_params = {'relationships':[('sbj','obj')], 'link_opacity':0.1, 'node_labels':ofv.nodeLabels()}
_rtg_ = rt.interactiveGraphPanel(ofv.df_triples, ln_params, w=1000, h=600)
if exists('../../../data/2014_vast/layout_example.parquet'): _rtg_.loadLayout('../../../data/2014_vast/layout_example.parquet')
# _rtg_

In [7]:
#rt.link(ofv.df_triples, **ln_params)
#_rtg_.selectEntities('Morlun',method='substring')
#_rtg_.selectedEntities()
#_rtg_.saveLayout('../../../data/2014_vast/layout_example.parquet')

In [8]:
# !pip install python-docx
import docx
import re

def findClosestEmail(filename):
    filename = filename.lower()
    best_match, best_count = None, 0
    for _email_ in emails_seen:
        username = _email_.split('@')[0]
        parts = re.split(' |\\-|\\.', username)
        part_count = 0
        for _part_ in parts:
            if _part_ in filename: part_count += 1
        if part_count > best_count: best_count, best_match = part_count, _email_
    return best_match

_lu_ = {'email':[], '__id_email__':[], 'resumefile':[], 'associated_entity':[], 'associated_entity_type':[]}
_base_resumes_dir_ = '../../../data/2014_vast/MC1/resumes/'
for _file_ in os.listdir(_base_resumes_dir_):
    _email_    = findClosestEmail(_file_)
    if _email_ is None: print(f'No email for {_file_}')
    _email_id_ = ofv.resolveUniqIdAndUpdateLookups(_email_, 'nmo:EmailAddress', 'uniq', 'sbj,obj')
    _docx_     = docx.Document(_base_resumes_dir_+_file_)
    for _para_ in _docx_.paragraphs:
        if len(_para_.text) > 0:
            _extracts_ = rt.textExtractEntities(_para_.text)
            for _extract_ in _extracts_:
                _lu_['email'].append(_email_)
                _lu_['__id_email__'].append(_email_id_)
                _lu_['resumefile'].append(_file_)
                _lu_['associated_entity'].append(_extract_[0])
                _lu_['associated_entity_type'].append(_extract_[1])
df_resumes = pl.DataFrame(_lu_)

In [9]:
ln_resume_params = {'relationships':[('email','associated_entity')], 'link_opacity':0.1, 'node_labels':ofv.nodeLabels()}
_g_resume_   = rt.createNetworkXGraph(df_resumes, ln_resume_params['relationships'])
_pos_        = rt.hyperTreeLayout(_g_resume_)
ln_resume_params['pos'] = _pos_
_rtg_resume_ = rt.interactiveGraphPanel(df_resumes, ln_resume_params, w=1000, h=600)
#_rtg_resume_

In [ ]:
_base_dir_ = '../../../data/2014_vast/MC1/News Articles'
_lu_ = {'__id_article__':[], 'directory':[], 'file':[], 'source':[], 'title':[], 'published':[], 'article':[], 'author':[], 'location':[]}
for _file_ in os.listdir(_base_dir_):
    _txt_raw_     = open(os.path.join(_base_dir_, _file_), 'rb').read()        
    _txt_better_  = str(_txt_raw_, encoding='latin').split('\n')
    _txt_         = []
    for _line_ in _txt_better_:
        _line_ = _line_.strip()
        if _line_ == '' or _line_ == '\n' or _line_ == '\r': continue
        _txt_.append(_line_)
    _source_           = _txt_[0]
    _title_            = _txt_[1]
    _author_           = _txt_[2]
    def numberInString(_string_): return any(char.isdigit() for char in _string_)
    if '/' in _author_ or numberInString(_author_): 
        _author_, _published_ = None, _author_
        _just_article_txt_    = _txt_[3:]
    else:               
        _published_           = _txt_[3]
        _just_article_txt_    = _txt_[4:]
    if 'drug' in _published_.lower():
        _published_ = None
        _just_article_txt_ = _txt_[3:]

    _location_ = None
    if ' - ' in _just_article_txt_[0][:50]:
        _possible_ = _just_article_txt_[0][:_just_article_txt_[0].find(' - ')].strip()
        if ',' in _possible_:
            _location_ = _possible_.lower()
            _just_article_txt_[0] = _just_article_txt_[0][_just_article_txt_[0].find(' - ')+3:].strip()
    _id_ = ofv.createId('NewsArticle')
    _lu_['__id_article__'].append(_id_)
    _lu_['directory'].append(_source_), _lu_['file'].append(_file_), _lu_['author'].append(_author_), _lu_['location'].append(_location_)
    _lu_['title'].append(_title_), _lu_['source'].append(_source_), _lu_['published'].append(_published_), _lu_['article'].append('\n'.join(_just_article_txt_))

df_articles = pd.DataFrame(_lu_)
df_articles = rt.columnsAreTimestamps(df_articles, 'published')
df_articles.sample(3)

In [ ]:
_base_ontology_name_ = '../../../data/2014_vast/MC1/ontology_rep/20240711_ontology'
ofv.to_files(_base_ontology_name_)
#ofv_load = rt.ontologyFrameworkInstance(base_filename=_base_ontology_name_)
#print(f'{len(ofv_load.df_triples)=} {len(ofv.df_triples)=}')
#print(f'{len(ofv_load.uid_lu)=} {len(ofv.uid_lu)=}')
#set(ofv_load.uid_lu) - set(ofv.uid_lu), set(ofv.uid_lu) - set(ofv_load.uid_lu)